In [1]:
import pandas as pd
import numpy as np
import matplotlib as mplot
import seaborn as sb
import time

In [2]:
movies_genres_data=pd.read_csv("Movies_genres.csv")

In [3]:
def numeric_eliminator(total_text, index, column):
        string = ""
        #print(total_text.split())
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalpha()))
            # Conver all letters to lower-case
            word = word.lower()
            string += word + " "
        movies_genres_data[column][index] = string

In [4]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in movies_genres_data.iterrows():
    numeric_eliminator(row['Title'], index, 'Title')
# we print the time it took to preprocess whole titles
print((time.clock() - start_time)/60, "minutes")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


6.176678787111492 minutes


In [5]:
movies_genres_data.to_csv("movies_data.csv",index=False)

# Content Based Recommendation System

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
movies=pd.read_csv('movies.csv')

In [4]:
tf = TfidfVectorizer(analyzer='word',min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['Genres'])

In [5]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
titles = movies['Title']
indices = pd.Series(movies.index, index=movies['Title'])

In [7]:
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [8]:
genre_recommendations('Good Will Hunting (1997)').head(20)

25                                        Othello (1995)
26                                   Now and Then (1995)
29     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
30                                Dangerous Minds (1995)
35                               Dead Man Walking (1995)
39                       Cry, the Beloved Country (1995)
42                                    Restoration (1995)
52                                       Lamerica (1994)
54                                        Georgia (1995)
56                          Home for the Holidays (1995)
61                             Mr. Holland's Opus (1995)
66                                       Two Bits (1995)
77                            Crossing Guard, The (1995)
79          White Balloon, The (Badkonake Sefid ) (1995)
81                       Antonia's Line (Antonia) (1995)
82       Once Upon a Time... When We Were Colored (1995)
89                    Journey of August King, The (1995)
92                             

In [9]:
genre_recommendations('Heat (1995)').head(20)

168                      Hackers (1995)
2209                       Ronin (1998)
2887    Someone to Watch Over Me (1987)
3694                         F/X (1986)
3695                       F/X 2 (1992)
1884      French Connection, The (1971)
2471              Corruptor, The (1999)
46                 Seven (Se7en) (1995)
49           Usual Suspects, The (1995)
517            Romeo Is Bleeding (1993)
653                  Purple Noon (1960)
1073              Reservoir Dogs (1992)
1331            Albino Alligator (1996)
1444            City of Industry (1997)
1601                 Playing God (1997)
1629                   Incognito (1997)
1640                  Red Corner (1997)
2140          Young and Innocent (1937)
2322              Simple Plan, A (1998)
2492                  True Crime (1999)
Name: Title, dtype: object

In [10]:
ratingsdata=pd.read_csv('ratings.csv')

In [11]:
ratingsdata.groupby('UserID').count()

,MovieID,Rating,Timestamp
UserID,,,
1,53,53,53
2,129,129,129
3,51,51,51
4,21,21,21
5,198,198,198
6,71,71,71
7,31,31,31
8,139,139,139
9,106,106,106


# Memory Based Collabarative Filtering

In [13]:
ratingdata=ratingsdata.reindex(np.random.permutation(ratingsdata.index))
small_data = ratingsdata.sample(frac=0.05)

In [14]:
small_data.head()

,UserID,MovieID,Rating,Timestamp
118400,765,1377,4,975612015
383470,2241,1103,4,974596312
196137,1206,435,2,974843978
329996,1947,2160,3,974692831
856882,5147,66,1,962022697


In [15]:
small_data.groupby('UserID').count()

,MovieID,Rating,Timestamp
UserID,,,
1,5,5,5
2,6,6,6
3,4,4,4
4,2,2,2
5,12,12,12
6,4,4,4
7,4,4,4
8,9,9,9
9,2,2,2


In [16]:
small_data.groupby('MovieID').count()

,UserID,Rating,Timestamp
MovieID,,,
1,95,95,95
2,36,36,36
3,16,16,16
4,9,9,9
5,25,25,25
6,50,50,50
7,16,16,16
8,2,2,2
9,4,4,4


In [17]:
small_data.shape

(50010, 4)

In [18]:
small_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50010 entries, 118400 to 478571
Data columns (total 4 columns):
UserID       50010 non-null int64
MovieID      50010 non-null int64
Rating       50010 non-null int64
Timestamp    50010 non-null int64
dtypes: int64(4)
memory usage: 1.9 MB


In [19]:
small_data=small_data[['UserID','MovieID','Rating']]

In [21]:
from sklearn.model_selection import train_test_split,cross_val_score
train_data, test_data = train_test_split(small_data, test_size=0.2,random_state=42)

In [23]:
train_data.shape

(40008, 3)

In [24]:
test_data.shape

(10002, 3)

In [25]:
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
